In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import pandas as pd
import time

driver = webdriver.Chrome()
wait = WebDriverWait(driver, 10)

KeyboardInterrupt: 

In [2]:
driver.get("http://www.cninfo.com.cn/new/commonUrl?url=disclosure/list/notice#szse%2Fimportant")

queryBox = driver.find_elements(By.TAG_NAME, "input")
queryBox[4].send_keys("董事会秘书")
driver.find_element(By.CLASS_NAME, "el-button--primary").click()
time.sleep(1.5)
# driver.close()

In [3]:
df = pd.DataFrame(columns=["code", "name", "title", "link", "time"])
while True:
    table = driver.find_element(By.CLASS_NAME, "el-table__body")
    for cell in table.find_elements(By.CLASS_NAME, "el-table__row"):

        # check title

        row = [
                cell.find_element(By.CLASS_NAME, "code").text, 
                cell.find_element(By.CLASS_NAME, "el-table_1_column_2").text, 
                cell.find_element(By.CLASS_NAME, "el-table_1_column_3").text, 
                cell.find_element(By.CLASS_NAME, "el-table_1_column_3").find_element(By.TAG_NAME, "a").get_attribute("href"), 
                cell.find_element(By.CLASS_NAME, "date").text
            ]
        df.loc[len(df)] = row
    next_page = driver.find_element(By.CLASS_NAME, "btn-next")
    if not next_page.get_attribute("disabled"):
        driver.find_element(By.CLASS_NAME, "btn-next").click()
        time.sleep(1.5)
    else:
        break

df


,code,name,title,link,time
0,301082,久盛电气,关于聘任董事会秘书、证券事务代表的公告,http://www.cninfo.com.cn/new/disclosure/detail...,2023-09-22
1,301525,儒竞科技,董事会秘书工作细则,http://www.cninfo.com.cn/new/disclosure/detail...,2023-09-21
2,002219,新里程,关于变更董事会秘书及证券事务代表的公告,http://www.cninfo.com.cn/new/disclosure/detail...,2023-09-19 07:56
3,301498,乖宝宠物,董事会秘书工作细则,http://www.cninfo.com.cn/new/disclosure/detail...,2023-09-19
4,301517,陕西华达,股东大会、董事会、监事会、独立董事、董事会秘书制度的建立健全及运行情况说明,http://www.cninfo.com.cn/new/disclosure/detail...,2023-09-18
...,...,...,...,...,...
574,002421,达实智能,《董事会秘书工作制度》（2023年3月）,http://www.cninfo.com.cn/new/disclosure/detail...,2023-03-22
575,001286,陕西能源,股东大会、董事会、监事会、独立董事、董事会秘书制度的建立健全及运行情况说明,http://www.cninfo.com.cn/new/disclosure/detail...,2023-03-21
576,001360,南矿集团,股东大会、董事会、监事会、独立董事、董事会秘书制度的建立健全及运行情况说明,http://www.cninfo.com.cn/new/disclosure/detail...,2023-03-21
577,000586,汇源通信,四川汇源光通信股份有限公司董事会秘书工作细则,http://www.cninfo.com.cn/new/disclosure/detail...,2023-03-21


In [42]:
driver.close()

In [9]:
df.to_excel("董秘公告v1.xlsx", index=False)

In [1]:
import pandas as pd
import jieba

df = pd.read_excel("董秘公告v1.xlsx")
df["time"] = df['time'].apply(pd.to_datetime)
df.head()

,code,name,title,link,time
0,301082,久盛电气,关于聘任董事会秘书、证券事务代表的公告,http://www.cninfo.com.cn/new/disclosure/detail...,2023-09-22 00:00:00
1,301525,儒竞科技,董事会秘书工作细则,http://www.cninfo.com.cn/new/disclosure/detail...,2023-09-21 00:00:00
2,2219,新里程,关于变更董事会秘书及证券事务代表的公告,http://www.cninfo.com.cn/new/disclosure/detail...,2023-09-19 07:56:00
3,301498,乖宝宠物,董事会秘书工作细则,http://www.cninfo.com.cn/new/disclosure/detail...,2023-09-19 00:00:00
4,301517,陕西华达,股东大会、董事会、监事会、独立董事、董事会秘书制度的建立健全及运行情况说明,http://www.cninfo.com.cn/new/disclosure/detail...,2023-09-18 00:00:00


In [4]:
def check(title):
    tokens = jieba.lcut(title, cut_all=True)
    format = [["董事","董事会"],
          ["秘书"],
          ["聘任", "变更", "代行", "辞职", "辞任"]]
    for key in format:
        flag = False
        for word in key:
            if word in tokens:
                flag = True
                break
        if not flag:
            return False
        
    return True

In [5]:
# filter old notices
df = df[df["time"] > pd.to_datetime("2023-08-01")]
df["related"] = df["title"].apply(check)
df = df[df["related"]].reset_index()
df

,level_0,index,code,name,title,link,time,related
0,0,0,301082,久盛电气,关于聘任董事会秘书、证券事务代表的公告,http://www.cninfo.com.cn/new/disclosure/detail...,2023-09-22 00:00:00,True
1,1,2,2219,新里程,关于变更董事会秘书及证券事务代表的公告,http://www.cninfo.com.cn/new/disclosure/detail...,2023-09-19 07:56:00,True
2,2,5,688,国城矿业,关于董事长代行董事会秘书职责的公告,http://www.cninfo.com.cn/new/disclosure/detail...,2023-09-16 00:00:00,True
3,3,7,300287,飞利信,关于董事会秘书任期届满离任及指定副总经理代行董事会秘书职责的公告,http://www.cninfo.com.cn/new/disclosure/detail...,2023-09-15 19:36:00,True
4,4,10,2830,名雕股份,关于董事会秘书、副总经理辞职的公告,http://www.cninfo.com.cn/new/disclosure/detail...,2023-09-14 00:00:00,True
...,...,...,...,...,...,...,...,...
73,74,145,839,ST国安,关于公司董事会秘书辞职的公告,http://www.cninfo.com.cn/new/disclosure/detail...,2023-08-04 00:00:00,True
74,75,148,301456,盘古智能,关于聘任副总经理、董事会秘书的公告,http://www.cninfo.com.cn/new/disclosure/detail...,2023-08-02 11:43:00,True
75,76,150,722,湖南发展,关于董事会秘书辞职的公告,http://www.cninfo.com.cn/new/disclosure/detail...,2023-08-02 00:00:00,True
76,77,151,548,湖南投资,湖南投资独立董事关于公司聘任董事会秘书的独立意见,http://www.cninfo.com.cn/new/disclosure/detail...,2023-08-02 00:00:00,True


In [33]:
df.to_excel("董秘公告v2.xlsx", index=False)

In [ ]:
import jieba
strs=["我来到北京清华大学","乒乓球拍卖完了","中国科学技术大学"]

seg_list = jieba.cut("我来到北京清华大学", cut_all=True)
print("Full Mode: " + "/ ".join(seg_list))  # 全模式

seg_list = jieba.cut("我来到北京清华大学", cut_all=False)
print("Default Mode: " + "/ ".join(seg_list))  # 精确模式

seg_list = jieba.cut("他来到了网易杭研大厦")  # 默认是精确模式
print(", ".join(seg_list))

seg_list = jieba.cut_for_search("小明硕士毕业于中国科学院计算所，后在日本京都大学深造")  # 搜索引擎模式
print(", ".join(seg_list))

Full Mode: 我/ 来到/ 北京/ 清华/ 清华大学/ 华大/ 大学
Default Mode: 我/ 来到/ 北京/ 清华大学
他, 来到, 了, 网易, 杭研, 大厦
小明, 硕士, 毕业, 于, 中国, 科学, 学院, 科学院, 中国科学院, 计算, 计算所, ，, 后, 在, 日本, 京都, 大学, 日本京都大学, 深造
